<a href="https://colab.research.google.com/github/caiocfpeixoto/VGG16_RANDOM_FOREST/blob/main/VGG16_%26_Random_Forest_V1_1_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import de pacotes necessários

In [ ]:
!pip install split-folders

In [ ]:
import numpy as np
import pandas as pd
import cv2
import seaborn as sns
import os
from PIL import Image

import matplotlib.pyplot as plt
import glob

from keras.models import Sequential
from keras.layers import Conv2D, Dense, Flatten, BatchNormalization, MaxPooling2D

from keras.applications import VGG16

from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier
from keras.utils import  to_categorical
from sklearn import metrics
from sklearn.metrics import confusion_matrix
import splitfolders

Conexão com Google drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
SIZE = 120
def datafunc(datadir):
    images = []
    labels = []
    file_names = []
    for dir_path in glob.glob(datadir):
        label = dir_path.split('/')[-1]
        for img_path in glob.glob(os.path.join(dir_path, '*.tif')):
            try:
                img = cv2.imread(img_path, cv2.IMREAD_COLOR)
                img = cv2.resize(img, (SIZE, SIZE))
                img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
                file_name = os.path.basename(img_path)
                file_names.append(file_name)
                images.append(img)
                labels.append(label)
            except Exception as e:
                print(e)

    return np.array(images), np.array(labels), np.array(file_names)

In [ ]:
seeds = ["12", "22", "32", "42", "52"]
vetor_resultados = []
base_path = "/content/drive/MyDrive/Celulas Cervicais_V1.1_seed_"
train_suffix = "/train/*"
test_suffix = "/test/*"

In [ ]:
VGG_model = VGG16(weights = 'imagenet', include_top = False, input_shape = (SIZE, SIZE, 3))

for layer in VGG_model.layers:
    layer.trainable = False

58889256/58889256 [==============================] - 1s 0us/step


In [ ]:
 RF_model = RandomForestClassifier(n_estimators=100, random_state=42)

In [ ]:
for seed in seeds:
  train_path = base_path + seed + train_suffix
  test_path = base_path + seed + test_suffix

  train_images, train_labels, train_images_names = datafunc(train_path)
  test_images, test_labels, test_images_names = datafunc(test_path)

  df = pd.DataFrame({'label': train_labels, 'images': list(train_images_names), 'path': list(train_images)}, columns=['label', 'images', 'path'])

  count_class_0, count_class_1 = df.label.value_counts()

  df_class_0 = df[df['label'] == 'NEGATIVA']
  df_class_1 = df[df['label'] == 'POSITIVA']

  df_class_0_under = df_class_0.sample(count_class_1)
  df_test_under = pd.concat([df_class_0_under, df_class_1], axis=0)

  t_images = []
  for img in df_test_under['path']:
      img = Image.fromarray(img)
      img = cv2.cvtColor(np.array(img), cv2.COLOR_RGB2BGR)
      t_images.append(img)
  t_images = np.array(t_images)
  train_labels = np.array(df_test_under['label'])

  le = preprocessing.LabelEncoder()
  le.fit(test_labels)
  test_labels_encoded = le.transform(test_labels)
  le.fit(train_labels)
  train_labels_encoded = le.transform(train_labels)

  x_train, y_train, x_test, y_test = t_images, train_labels_encoded, test_images, test_labels_encoded
  x_train, x_test = x_train / 255.0, x_test / 255.0 #scaling the data

  VGG_model = VGG16(weights = 'imagenet', include_top = False, input_shape = (SIZE, SIZE, 3))

  for layer in VGG_model.layers:
      layer.trainable = False

  feature_extractor = VGG_model.predict(x_train)
  features = feature_extractor.reshape(feature_extractor.shape[0], -1)

  X_rf = features

  RF_model = RandomForestClassifier(n_estimators=100, random_state=42)

  RF_model.fit(X_rf, y_train)

  X_test_feature = VGG_model.predict(x_test)
  X_test_features = X_test_feature.reshape(X_test_feature.shape[0], -1)

  prediction_RF = RF_model.predict(X_test_features)

  cm = confusion_matrix(y_test, prediction_RF)
  sns.heatmap(cm, annot = True)

  tp, fp, fn, tn = cm.ravel()

  accurary = metrics.accuracy_score(y_test, prediction_RF)
  recall = metrics.recall_score(y_test, prediction_RF)
  precision = metrics.precision_score(y_test, prediction_RF)
  kappa = metrics.cohen_kappa_score(y_test, prediction_RF)

  metricas = (accurary, recall, precision, kappa)

  vetor_resultados.append(metricas)

print(vetor_resultados)

72/72 [==============================] - 345s 5s/step


In [ ]:
dataf = pd.DataFrame(vetor_resultados, columns=['Accuracy', 'Recall', 'Precision', 'Kappa'])

# Crie um boxplot para as colunas do DataFrame
df.boxplot(column=['Accuracy', 'Recall', 'Precision', 'Kappa'])

# Rotule o eixo y
plt.ylabel('Valor')

# Defina o título do gráfico
plt.title('Boxplot das Métricas')

# Exiba o gráfico
plt.show()